<a href="https://colab.research.google.com/github/dingzhang2023/problem-solving-practice/blob/colab/Trie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Trie

**04/07/2024**

#### Core idea

> -  Group the string according to their prefixes, where strings with the same first letter are grouped together, and so on.
> -  To ensure that each letter corresponds to a specific node in a tree where each node's parent is the previous node's corresponding letter, `s[i]` is the parent node of `s[i + 1]`.
> - Example of trie, a trie containing the words `apple`, `app`, `banana`, `bat` and `cat`. Here's how the Trie looks:

                  (root)
                  /
                a
                / \
              p   b
              / \   \
            p   p   a
            /     \   \
          l       l   t
          /         \
        p           e
        /
        p
        |
        e

> - Save extra information in each node based on problems, such as `is_word` and `cnt`.


#### Major use cases
The main problems solved by Trie include:

1. String problems related to prefix
2. XOR Number problems, which can be converted to binary representation using trie


#### Trie code template
[208. Implement Trie (Prefix Tree)](https://leetcode.com/problems/implement-trie-prefix-tree/description/)
> Trie template problem
> - Insert
> - Query
> - Extra information saved in each node of trie, for this problem, flag `is_word` saved for id this node is a word or not.

In [ ]:
class Node:
    __slot__ = 'son', 'is_word'


    def __init__(self):
        self.son = defaultdict(Node)
        self.is_word = False


class Trie:

    def __init__(self):
        self.root = Node()


    def insert(self, word: str) -> None:
        cur = self.root
        for c in word:
            cur = cur.son[c]
        cur.is_word = True

    def search(self, word: str) -> bool:
        cur = self.root
        for c in word:
            if c not in cur.son:
                return False
            cur = cur.son[c]
        return cur.is_word

    def startsWith(self, prefix: str) -> bool:
        cur = self.root
        for c in prefix:
            if c not in cur.son:
                return False
            cur = cur.son[c]
        return True

#### Problem lists

[2416. Sum of Prefix Scores Strings](https://leetcode.com/problems/sum-of-prefix-scores-of-strings/)

>

[3045. Count Prefix and Suffix Pairs II](https://leetcode.com/problems/count-prefix-and-suffix-pairs-ii/description/)

**Method I**
> To transform the problem of determining whether one string is a prefix of another into a problem of only checking for prefixes

> - Convert `s` to a list of pairs: [(s[0], s[n-1]), (s[1], s[n-2]),...,(s[n-1], s[0])]
> - Check whether the pair list associated with `words[i]` is a prefix of the pair list associated with `words[j]`
> - Use trie to check the prefix

In [ ]:
class Node:
  __slot__ = 'son', 'cnt'

  def __init__(self):
    self.son = dict() # key is pair, value is node
    self.cnt = 0 # the count of the words(pair) ends with this node

class Solution:
    def countPrefixSuffixPairs(self, words: List[str]) -> int:
        ans = 0
        root = Node()
        for s in words:
            cur = root
            for p in zip(s, s[::-1]):
                # p = (s[i], s[n-1-i])
                if p not in cur.son:
                    cur.son[p] = Node()
                cur = cur.son[p]
                ans += cur.cnt
            # update the count of pair ends with s
            cur.cnt += 1
        return ans

Method II
> If `s` is both a prefix and a suffix of `t`, then for `t`, the length `|S|` of its prefix and suffix must be the same. The Z-function is defined as z[i] = LCP(s[i:], s) = n - i
>
